%%% Imports

In [1]:

from __future__ import annotations
from time import time
from BrisbaneVPRDatasetSpeed import BrisbaneVPRDatasetSpeed
from VPRNetwork import VPRNetwork
from plotting import plot_confusion_matrix, plot_match_images, plot_gps
from constants import brisbane_event_traverses, brisbane_event_traverses_aliases
from utils import get_short_traverse_name

import os, sys
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy import signal
from datetime import datetime

import numpy as np

import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer.classifier import Rate


In [2]:
# Training settings
from cgi import test


epochs = 50
batch_size = 10

# Network settings
input_size = 34
threshold = 1.0

# Data settings
train_traverse = brisbane_event_traverses[0]
test_traverse = brisbane_event_traverses[1]
train_name = brisbane_event_traverses_aliases[get_short_traverse_name(train_traverse)]
test_name = brisbane_event_traverses_aliases[get_short_traverse_name(test_traverse)]
num_places = 30
start_time = 50
place_gap = 10 #164/num_places # The streams run for approximately 164 seconds 
samples_per_sec = 1000
place_duration = 2
max_spikes = None

# Sequencer settings
sequence_length = 3

# Plot settings
redo_plot = False
vmin = 0
vmax = 0.05

def transpose( matrix):
    if len(matrix) == 0:
        return []
    return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]


# Make a folder for the trained network
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# Use GPU
print(torch.cuda.is_available())
device = torch.device('cuda')

#---------------- Create the Network -----------------#

# Create the network
net = VPRNetwork(input_size, num_places, threshold=threshold).to(device)


True


In [3]:
# Load the data
training_set = BrisbaneVPRDatasetSpeed(train_traverse, train=True, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time,samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size)
testing_set  = BrisbaneVPRDatasetSpeed(test_traverse, train=False, place_duration = place_duration, training_locations=training_set.place_locations, place_gap=place_gap, num_places=num_places, start_time=start_time, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size)
            
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=True)

Loading training event streams ...
Adding GPS
Adding GPS


  0%|          | 0/1 [00:00<?, ?it/s]

Duration: 527.16s (which is 267028797 events)


/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['x'].loc[small_filt0x] = i
/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['y'].loc[small_filt0y] = i
/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

[[4.9440e+01 5.0000e+01]
 [4.7346e+01 6.0000e+01]
 [4.5667e+01 7.0000e+01]
 [3.0662e+01 8.0000e+01]
 [5.3345e+01 9.0000e+01]
 [5.2011e+01 1.0000e+02]
 [5.8582e+01 1.1000e+02]
 [5.4396e+01 1.2000e+02]
 [5.6436e+01 1.3000e+02]
 [5.1793e+01 1.4000e+02]
 [3.6209e+01 1.5000e+02]
 [6.3223e+01 1.6000e+02]
 [6.1511e+01 1.7000e+02]
 [6.1192e+01 1.8000e+02]
 [4.7303e+01 1.9000e+02]
 [5.7037e+01 2.0000e+02]
 [5.6466e+01 2.1000e+02]
 [5.6813e+01 2.2000e+02]
 [5.7803e+01 2.3000e+02]
 [5.7392e+01 2.4000e+02]
 [5.4293e+01 2.5000e+02]
 [4.9978e+01 2.6000e+02]
 [1.4703e+01 2.7000e+02]
 [5.7440e+00 2.8000e+02]
 [3.9436e+01 2.9000e+02]
 [2.1800e-01 3.0000e+02]
 [0.0000e+00 3.1000e+02]
 [0.0000e+00 3.2000e+02]
 [8.5990e+00 3.3000e+02]
 [0.0000e+00 3.4000e+02]]
The number of training substreams is: 30
Loading testing event streams ...
Adding GPS
Adding GPS
[[ 50.12152  49.72   ]
 [ 52.865    59.     ]
 [ 50.3947   68.15   ]
 [ 29.96344  78.54   ]
 [ 54.734    88.5    ]
 [ 55.2648   97.85   ]
 [ 58.35954 10

  0%|          | 0/1 [00:00<?, ?it/s]

Duration: 360.22s (which is 266579839 events)


/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['x'].loc[small_filt0x] = i
/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['y'].loc[small_filt0y] = i
/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

The number of testing substreams is: 30


In [4]:
# Define an optimiser 
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training the network
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)

# Create a training assistant object
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)


if redo_plot:
    for i, (input, label) in enumerate(train_loader): # training loop
        output = assistant.train(input, label)

else: 
    for epoch in range(epochs):

        for i, (input, label) in enumerate(train_loader): # training loop
            output = assistant.train(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        for i, (input, label) in enumerate(test_loader): # training loop
            output = assistant.test(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        if epoch%20 == 19: # cleanup display
            print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {stats}'))
            stats_str = str(stats).replace("| ", "\n")
            print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')

        if stats.testing.best_accuracy:
            torch.save(net.state_dict(), trained_folder + '/network.pt')
        stats.update()
        stats.save(trained_folder + '/')
        net.grad_flow(trained_folder + '/')

[Epoch  0/50] Train loss =     1.39258 | Test  loss =     0.79423                        accuracy = 0.03333
Accuracy was none

Accuracy was none
[Epoch  1/50] Train loss =     0.57364 (min =     1.39258)    accuracy = 0.16667  | Test  loss =     0.71526 (min =     0.79423)    accuracy = 0.10000 (max = 0.03333)
Accuracy was none
[Epoch  2/50] Train loss =     0.56291 (min =     0.57364)    accuracy = 0.16667 (max = 0.16667) | Test  loss =     0.68888 (min =     0.71526)    accuracy = 0.20000 (max = 0.10000)
Accuracy was none
[Epoch  3/50] Train loss =     0.50396 (min =     0.56291)    accuracy = 0.20000 (max = 0.16667) | Test  loss =     0.94060 (min =     0.68888)    accuracy = 0.20000 (max = 0.20000)
Accuracy was none
[Epoch  4/50] Train loss =     0.54190 (min =     0.50396)    accuracy = 0.20000 (max = 0.20000) | Test  loss =     0.59657 (min =     0.68888)    accuracy = 0.40000 (max = 0.20000)
Accuracy was none
[Epoch  5/50] Train loss =     0.41491 (min =     0.50396)    accuracy

In [5]:
# import the best network during training 
net.load_state_dict(torch.load(trained_folder + '/network.pt'))
net.export_hdf5(trained_folder + '/network.net')

# Get the output for the input to each place
test_loader2  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=False)
rate = []
labels = []
for i, (input, label) in enumerate(test_loader2):
    output = net(input.to(device)) # Get network output
    #guesses = assistant.classifier(output).cpu().data.numpy() # get the predictions 
    rate.extend(Rate.rate(output).cpu().data.numpy()) # Get the firing rates for each place 
    labels.extend(label.cpu().data.numpy()) # Get place labels


# # Get rates in percentages of total
# for i in range(num_places):
#     sum = np.sum(rate[i])
#     if sum != 0:
#         rate[i] = np.divide(rate[i],sum)

# Make confusion matrix annotations
accuracy = 0
matches = []
annotations = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(rate[qryIndex])
    matches.append(max_idx)
    annotations[max_idx][qryIndex] = 'x'
    if max_idx ==qryIndex:
        accuracy += 1


#--------------- Apply a sequencer ------------------#
I = np.identity(sequence_length)
conv = signal.convolve2d(rate, I, mode='same')
print(np.shape(conv))

# Make confusion matrix annotations
accuracy_s = 0
matches_with_seq = []
annotations_s = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(conv[qryIndex])
    matches_with_seq.append(max_idx)
    annotations_s[max_idx][qryIndex] = 'x'
    if max_idx == qryIndex:
        accuracy_s += 1



(30, 30)


In [6]:

#--------------- Save Results ------------------#

# Make new folder for results
from tracemalloc import start
from plotting import plot_gps


time_stamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_path = "./../results/" + time_stamp
os.mkdir(results_path)


# Save query and match images 
images_path = results_path + "/matched_images"
images_path_seq = results_path + "/matched_images_seq"
os.mkdir(images_path)
os.mkdir(images_path_seq)

print('help')
plot_match_images(images_path + "/", matches, training_set.place_images, testing_set.place_images)
plot_match_images(images_path_seq + "/", matches_with_seq, training_set.place_images, testing_set.place_images)

# Save the confusion matrices
confusion_path = results_path + "/confusion_matrices"
os.mkdir(confusion_path)
rate = transpose(rate)
conv = transpose(conv)
output_path = confusion_path + "/confusion_matrix" 
output_path_s = confusion_path + "/confusion_matrix_seq" 
plot_confusion_matrix(rate, labels, annotations, output_path, vmin, vmax)
plot_confusion_matrix(conv, labels, annotations_s, output_path_s, vmin, vmax)

# Save GPS map'
gps_path = results_path + "/gps_locations"
plot_gps(gps_path, training_set.place_locations, testing_set.place_locations)

# Save test settings and accuracy
accuracy = accuracy/num_places
accuracy_s = accuracy_s/num_places


help


<Figure size 1296x288 with 0 Axes>

<Figure size 1296x288 with 0 Axes>

In [7]:

log_path = results_path + "/log.txt"
log_string  = """
---- DATA SETTINGS ---- 
Training datasets = {}
Testing datasets = {} 
# Places = {}
Start time = {} [s]
Place gap = {} [s]
Place duration = {} [s]
Samples per second = {} 
Max spikes per sample = {}
---- NETWORK SETTINGS ---- 
Input size = {}x{} 
Threshold = {}
---- TRAINING SETTINGS ---- 
Epochs = {}
Batch size = {}
---- SEQUENCER SETTINGS ----
Sequence lengtth = {} 
---- RESULTS ----
Accuracy = {}
Accuracy (sequencer) = {}
""".format(train_name,test_name,num_places,start_time,place_gap,place_duration,
                samples_per_sec,max_spikes,input_size,input_size,threshold,epochs,
                batch_size,sequence_length,accuracy,accuracy_s)
f = open(log_path,'w')
f.write(log_string)
f.close()

print("The accuracy of the network is: " + str(accuracy))
print("The accuracy with a sequencer is: " + str(accuracy_s))

The accuracy of the network is: 0.7333333333333333
The accuracy with a sequencer is: 0.8666666666666667
